In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, clear_output


## Importing Dataset

In [2]:
df= pd.read_csv("movies_metadata.csv")
df['overview'] = df['overview'].fillna('')
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


## Baseline Implementation

## Item Profile

In [3]:
tf = TfidfVectorizer(stop_words='english')
tfm = tf.fit_transform(df['overview'])


In [4]:
def content_recommendation (movie_name, n=10):
    movie_series = pd.Series(df.index, index=df['title']).drop_duplicates()
    movie_index = movie_series [movie_name]
    cosimi = linear_kernel (tfm[movie_index],tfm) 
    movie_list = list (movie_series.index)
    len(movie_list)
    cosimi_df = pd.DataFrame()
    cosimi_df['title'] =  movie_list
    cosimi_df['cosimi_score']= cosimi[0]
    cosimi_df
    cosimi_top =cosimi_df.sort_values('cosimi_score',ascending=False)
    return pd.DataFrame( cosimi_top.iloc[1:].head(n))

#### Item profile for the movie "The Godfather" : 
Movies that are most similar to  "The Godfather"

In [5]:
movie_name= "The Godfather"
n=7  ## Number of recommendation required

content_recommendation (movie_name, n)

,title,cosimi_score
1178,The Godfather: Part II,0.475954
44030,The Godfather Trilogy: 1972-1990,0.356142
1914,The Godfather: Part III,0.183049
23126,Blood Ties,0.153702
11297,Household Saints,0.140767
34717,Start Liquidation,0.134386
10821,Election,0.132326


From above results we can clearly see movie related to "The Godfather" series such as "The Godfather: Part II" and "The Godfather: Part III" are recommended. Crime genre movies similar to "The Godfather" such as "Blood Ties" is also shown in the result.

## User Profile

In [6]:

ratings_df = pd.read_csv("ratings_small.csv")
ratings_df 

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [24]:

def user_movie_recommendation(uid, df, ratings_df):
    user_ratings_df = ratings_df[ratings_df.userId == uid]
    user_ratings_df 
    df['id']=df['id'].astype(str)
    user_ratings_df['movieId']=user_ratings_df['movieId'].astype(str)
    user_ratings_df = pd.merge(user_ratings_df, df, left_on='movieId', right_on='id', left_index=True)
    user_ratings_df = pd.DataFrame(user_ratings_df [["movieId","title","rating","overview"]])
    user_ratings_df 

    user_ratings_df ["normalized_rating"] = user_ratings_df['rating']/5
    user_ratings_df
    tfm_user = tfm[user_ratings_df.index].toarray()
    array1 = np.dot (tfm_user.T,user_ratings_df["normalized_rating"].values)
    user_cosim = cosine_similarity(np.atleast_2d(array1), tfm)
    user_cosim_df = pd.DataFrame(user_cosim.T)
    result = pd.concat([df[["id","title"]], user_cosim_df ], axis=1)
    result = result.rename({0 : 'user_predicted_rating'}, axis=1)
    result =result.sort_values('user_predicted_rating',ascending=False)
    result["user_predicted_rating"]=5*result["user_predicted_rating"]
    return result 




In [31]:
uid = 1
result_df = user_movie_recommendation(uid, df, ratings_df)
result_df.head(10)

,id,title,user_predicted_rating
2591,2105,American Pie,3.380987
3213,2193,My Tutor,2.019848
2297,1371,Rocky III,1.991073
10719,2455,Confidentially Yours,1.981087
4604,2294,Jay and Silent Bob Strike Back,1.607900
23729,253310,Premature,1.212664
22783,99367,Date and Switch,1.079885
23636,70706,Very Good Girls,1.054001
30465,14912,18 Year Old Virgin,1.009826
18274,61896,Prom Night in Mississippi,0.959160


## Error Calculation : MSE 

In [10]:
from sklearn.utils import shuffle

num_rows = ratings_df.shape[0]
train, test =ratings_df.copy(deep = True),  ratings_df[int(num_rows*0.8):].copy(deep = True)
train[int(num_rows*0.8):]['rating']  = train[int(num_rows*0.8):]['rating']*0
test =shuffle(test)
test["movieId"]=test["movieId"].astype(str)
test

,userId,movieId,rating,timestamp
83014,564,338,2.0,974844797
98666,662,186,3.0,839022731
83545,564,1438,1.0,974839481
89307,595,2858,5.0,965077208
97111,650,780,5.0,844883683
...,...,...,...,...
82261,561,2710,2.0,1172696885
81885,558,4995,5.0,1026053672
99694,666,10,3.0,838920928
97800,654,2890,4.5,1145390589


In [22]:
test = test.sort_values('userId')
test_users_list = test['userId'].unique().tolist()
print (len(test_users_list))
count = 0
error =0
misses = 0

ntul= len(test_users_list)
for j in range (ntul):
    clear_output(wait=True)
    display (str(j+1)+" / "+str(ntul) )
    uid = test_users_list[j]
    test_user = test[test.userId==uid]
    result_user_df =  user_movie_recommendation(uid, df, ratings_df)
    result_user_movie_list = result_user_df["id"].tolist()
    
    #print (uid, "    ", test_user.shape[0])
    count = count + test_user.shape[0]
    for i in range(test_user.shape[0]):
        if test.iloc[i]["movieId"] in result_user_movie_list:
            #print(test.iloc[i]["movieId"])
            y= test.iloc[i]["rating"]
            y_hat == float (result_user_df[result_user_df.id == test.iloc[i]["movieId"]]['user_predicted_rating'])
            #print(y_hat)
            error  = error + (y-y_hat)**2
        else:
            misses =misses+1
            y= test.iloc[i]["rating"]
            y_hat = 0
            error  = error + (y-y_hat)**2
        #print(error)



'125 / 125'

In [23]:
print ("Total test values =", count)
print("Misses =", misses)
print("Sum of Squared Error =", error)
print("Mean Squared Error =", error/count)

Total test values = 20001
Misses = 15260
Sum of Squared Error = 264858.25
Mean Squared Error = 13.242250387480626
